In [ ]:
# Install vector database
# ! pip install vectordb

## Import modules


In [ ]:
import os
import numpy as np
from tqdm.notebook import trange, tqdm
from PIL import Image, ImageFont, ImageDraw
import torch
import clip
import json as js
from docarray import DocList, BaseDoc
from docarray.typing import NdArray
import numpy as np
from vectordb import InMemoryExactNNVectorDB, HNSWVectorDB
from IPython.display import clear_output, display, HTML
from natsort import natsorted
import pandas as pd
from typing import List
import webbrowser
import shutil
import random
from utils import create_html_script

## Constants


In [ ]:
MODEL = "ViT-B/32"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

METADATA_PATH = "../data/metadata/"
KEYFRAME_PATH = "../data/keyframes/"
FEATURE_PATH = "../data/features/"
MAP_KEYFRAMES = "../data/map-keyframes/"
VIDEOS_PATH = "../data/videos/"
SCRIPT_PATH = "../data/scripts/"

WORKSPACE = "./vectordb"

In [ ]:
# verify scripts
print(len(os.listdir(VIDEOS_PATH)))
print(len(os.listdir(SCRIPT_PATH)))
print(len(os.listdir(VIDEOS_PATH)) == len(os.listdir(SCRIPT_PATH)))

## Re-formating Dataset


#### Get video names


In [ ]:
LEN_OF_KEYFRAME_NAME = 4

In [ ]:
video_names = [name for name in os.listdir(KEYFRAME_PATH) if name != ".gitkeep"]
print(video_names)

In [ ]:
for name in video_names:
    keyframes = [path for path in os.listdir(os.path.join(KEYFRAME_PATH, name))]
    for kf in keyframes:
        img_name = kf.split(".")[0]
        if len(img_name) != LEN_OF_KEYFRAME_NAME:
            changed_path = os.path.join(KEYFRAME_PATH, name, img_name.zfill(4) + ".jpg")
            old_path = os.path.join(KEYFRAME_PATH, name, kf)
            print(f"Change {old_path} to {changed_path}")
            os.rename(old_path, changed_path)

## Text embedding


In [ ]:
class TextEmbedding:
    def __init__(self):
        self.device = DEVICE
        self.model, _ = clip.load(MODEL, device=self.device)

    def __call__(self, text: str) -> np.ndarray:
        text_inputs = clip.tokenize([text]).to(self.device)
        with torch.no_grad():
            text_feature = self.model.encode_text(text_inputs)[0]
        return text_feature.detach().cpu().numpy()

    def __call__(self, texts) -> np.ndarray:
        text_inputs = clip.tokenize(texts).to(self.device)
        with torch.no_grad():
            text_feature = self.model.encode_text(text_inputs)[0]
        return text_feature.detach().cpu().numpy()

In [ ]:
text_embedding = TextEmbedding()

In [ ]:
query = "A blue sky in the background"
query_feat = text_embedding(query)
print(len(query_feat))

In [ ]:
querys = [
    "A blue sky in the background",
    "People hangout at the beach",
    "Birds are flying in the sky",
]
querys_feat = text_embedding(querys)
print(len(querys_feat))

## Vector Database


### Frame Document Class


In [ ]:
class FrameDoc(BaseDoc):
    embedding: NdArray[512]
    video_name = ""
    image_path = ""
    keyframe_id = 0
    actual_idx = 0
    actual_time = 0.0
    metadata = {}

    def __str__(self):
        return f"""
            Video name: {self.video_name}
            Image path: {self.image_path}
            Keyframe Id: {self.keyframe_id}
            Actual keyframe idx: {self.actual_idx}
            Time: {self.actual_time}
            Metadata: {self.metadata}
          """

## Database Handler


In [ ]:
class VectorDB:
    text_embedding = TextEmbedding()
    workspace = os.getcwd()
    method = "ANN"

    def __init__(self, method="ANN"):
        # Check if parent workspace exists
        if not os.path.isdir(WORKSPACE):
            os.mkdir(WORKSPACE, 0o666)
        # Create new workspae
        exits = [int(name.rsplit("_")[1]) for name in os.listdir(WORKSPACE)]
        while True:
            id = random.getrandbits(128)
            if id not in exits:
                self.workspace = os.path.join(
                    self.workspace, WORKSPACE, "DB_" + str(id)
                )
                break

        self.method = method
        #   Approximate Nearest Neighbour based on HNSW algorithm
        if method == "ANN":
            self.DB = HNSWVectorDB[FrameDoc](workspace=self.workspace)

        # Exhaustive search on the embeddings
        else:
            self.DB = InMemoryExactNNVectorDB[FrameDoc](workspace=self.workspace)

    def index(self, doc_list: List[FrameDoc]):
        # Index database
        self.DB.index(inputs=DocList[FrameDoc](doc_list))

    def search(self, query_text: str, topk=100):
        query_doc = FrameDoc(embedding=self.text_embedding(query_text))
        return self.DB.search(inputs=DocList[FrameDoc]([query_doc]), limit=topk)[
            0
        ].matches

    def delete(self, del_doc_list: List[FrameDoc]):
        self.DB.delete(docs=DocList[FrameDoc](del_doc_list))

### Needed functions


### Get all features files


In [ ]:
def get_all_feats():
    return [
        os.path.join(FEATURE_PATH, file)
        for file in os.listdir(FEATURE_PATH)
        if file.endswith(".npy")
    ]

In [ ]:
all_feat_files = get_all_feats()
print(all_feat_files)
print(len(all_feat_files))

### Create all the Docs


In [ ]:
def get_all_docs(npy_files):
    doc_list = []
    for feat_npy in npy_files:
        video_name = feat_npy[feat_npy.find("L") :].split(".")[0]
        feats_arr = np.load(os.path.join(feat_npy))
        # Load metadata
        metadata = {}
        with open(os.path.join(METADATA_PATH, video_name + ".json")) as meta_f:
            metadata = js.load(meta_f)
            map_kf = pd.read_csv(
                os.path.join(MAP_KEYFRAMES, video_name + ".csv"),
                usecols=["pts_time", "frame_idx"],
            )
            metadata = {key: metadata[key] for key in ["publish_date", "watch_url"]}
            for frame_idx, feat in enumerate(feats_arr):
                image_path = os.path.join(
                    KEYFRAME_PATH, video_name, f"{frame_idx + 1:04d}.jpg"
                )
                # frame_metadata = metadata.copy()
                # frame_metadata["watch_url"] = frame_metadata["watch_url"]  + "&t=" + str(map_kf["pts_time"][frame_idx]) + "s"
                actual_idx = map_kf["frame_idx"][frame_idx]
                doc_list.append(
                    FrameDoc(
                        embedding=feat,
                        video_name=video_name,
                        image_path=image_path,
                        keyframe_id=frame_idx + 1,
                        actual_idx=actual_idx,
                        actual_time=map_kf["pts_time"][frame_idx],
                        metadata=metadata,
                    )
                )

    return doc_list

In [ ]:
doc_list = get_all_docs(all_feat_files)

In [ ]:
print(len(doc_list))
print(doc_list[100])

## Utils


### !!! Vip pro UI


In [ ]:
def FrameDocToImage(docs):
    return [
        {
            "link": doc.metadata["watch_url"].split("v=")[-1],
            "path": doc.image_path,
            "video": doc.video_name,
            "frame": doc.actual_idx,
            "s": str(int(doc.actual_time) // 60)
            + "'"
            + str(round(doc.actual_time - 60 * (int(doc.actual_time) // 60), 1)),
        }
        for doc in docs
    ]

In [ ]:
def visualize(docs):
    display(HTML(create_html_script(FrameDocToImage(docs))))

### Open video at specific times


In [ ]:
# def open_video(doc: FrameDoc):
#     webbrowser.open(doc.metadata["watch_url"])

## Visualization functions


In [ ]:
# def get_images(results):
#     images = []
#     for i, res in enumerate(results):
#         img = Image.open(res.image_path)
#         draw = ImageDraw.Draw(img)
#         font = ImageFont.truetype("arial.ttf", 50)
#         draw.text(xy=(5, 5), text=f"{i}, {res.video_name}, {res.actual_idx}", align="left", fill=(255,0,0,255), font=font)
#         images.append(img)
#     return images

In [ ]:
# def visualize(imgs: List[Image.Image]) -> None:
#     rows = len(imgs) // 2 # see more clearly
#     if not rows:
#         rows += 1
#     cols = len(imgs) // rows
#     if rows * cols < len(imgs):
#         rows += 1
#     w, h = imgs[0].size
#     grid = Image.new('RGB', size=(cols * w, rows * h))
#     grid_w, grid_h = grid.size

#     for i, img in enumerate(imgs):

#         grid.paste(img, box=(i % cols * w, i // cols * h))

#     display(grid)

## DEMO


### Create DB


In [ ]:
DB = VectorDB()
DB.index(doc_list)

### Query


#### Query text


Đoạn video về một chiếc xe 7 chỗ biển số màu vàng hiệu Toyota tông vào gốc cây bên lề đường. Bên cạnh chiếc xe có biển báo giao nhau với đường không ưu tiên. Trong đoạn video có cảnh một người bảo vệ áo màu xanh đang nằm trên chiếc xe máy.


### Filter 1


In [ ]:
results1 = DB.search(
    "Three men are running. On the left of the frame is a row of small green trees next to scattered items, and on the right is a carpet of grass.",
    1000,
)  # Nên lấy nhiều

In [ ]:
round(3.12, 2)

In [ ]:
visualize(results1[:100])

### Filter 2


In [ ]:
DB2 = VectorDB()
DB2.index(results1)

In [ ]:
results2 = DB2.search("A bag is on the road", 500)

In [ ]:
visualize(results1[:50])